# working model for tensorfusion

In [55]:
import pickle
import torch
from torchmetrics import F1Score
import pandas as pd
import numpy as np
from glob import glob
from collections import Counter
import torch.nn.functional as F
from tqdm import tqdm
import os
from datasets import Dataset
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import random

print(torch.__version__)

1.13.1+cu117


## wav, text데이터 불러오기

In [56]:
# wav and text data load
dataset_file_lst = ['../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY19_annotation_nonmissing.pkl',
                    '../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY19_embedding_for_dataset.pkl',
                    '../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY20_annotation_nonmissing.pkl',
                    '../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY20_embedding_for_dataset.pkl']
dataset_file_lst = sorted(dataset_file_lst)
dataset_file_lst

['../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY19_annotation_nonmissing.pkl',
 '../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY19_embedding_for_dataset.pkl',
 '../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY20_annotation_nonmissing.pkl',
 '../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY20_embedding_for_dataset.pkl']

In [57]:

var_names = ['kemdy19_annot', 'kemdy19_emb', 'kemdy20_annot', 'kemdy20_emb']
for file, var_name in zip(dataset_file_lst, var_names):
    print(file, var_name)
    with open(file, 'rb') as f:
        globals()[var_name] = pickle.load(f)

../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY19_annotation_nonmissing.pkl kemdy19_annot
../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY19_embedding_for_dataset.pkl kemdy19_emb
../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY20_annotation_nonmissing.pkl kemdy20_annot
../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY20_embedding_for_dataset.pkl kemdy20_emb


In [58]:
#  paradeigma가 섞어놓은 label들..되돌리기
# {'neutral': 0, 'happy': 0, 'surprise':0, 'disgust': 0, 'angry': 0, 'sad':0, 'fear': 0}
# {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}

def change_label_order(lst, new_order = [4, 3, 6, 1, 0, 5, 2]):
    return [lst[i] for i in new_order]

kemdy20_annot['emotion_vector'] = kemdy20_annot['emotion_vector'].apply(change_label_order)
kemdy19_annot['emotion_vector'] = kemdy19_annot['emotion_vector'].apply(change_label_order)

In [59]:
kemdy19_annot.head(3)

,Segment ID,Emotion,Valence,Arousal,emotion_vector,valence_vector,arousal_vector,EDA,TEMP,EDA length,TEMP length,Scaled EDA,Scaled TEMP
0,Sess01_script01_M001,surprise,1.7,4.0,"[0, 0, 1, 0, 0, 0, 9]","[4, 5, 1, 0, 0]","[1, 0, 0, 6, 3]","[4.408951, 4.403825, 4.410233, 4.421767, 4.429...","[34.66, 34.66, 34.66, 34.66, 34.66, 34.68, 34....",33,33,"[-0.926455937246618, -0.9285481420682529, -0.9...","[1.6900874192514124, 1.6900874192514124, 1.690..."
1,Sess01_script01_F001,fear,1.5,3.6,"[0, 0, 7, 0, 1, 0, 2]","[5, 5, 0, 0, 0]","[1, 0, 2, 6, 1]","[0.145914, 0.140794, 0.144634, 0.145914, 0.140...","[30.69, 30.69, 30.69, 30.69, 30.71, 30.71, 30....",17,17,"[-1.407155706494133, -1.4312942323175069, -1.4...","[-1.1135528407658513, -1.1135528407658513, -1...."
2,Sess01_script01_M002,angry,1.3,4.3,"[10, 0, 0, 0, 0, 0, 0]","[7, 3, 0, 0, 0]","[1, 0, 0, 3, 6]","[4.478828, 4.396809, 4.334012, 4.322478, 4.346...","[34.61, 34.61, 34.61, 34.61, 34.61, 34.61, 34....",27,27,"[-0.8979352590723317, -0.9314117606848334, -0....","[1.6164861115016742, 1.6164861115016742, 1.616..."


<2020년>
'angry;disqust',  'angry;disqust;fear;neutral;sad',  'angry;disqust;neutral', 'angry;happy;neutral', 'angry;neutral', 'disqust;happy;neutral', 'disqust;neutral', 'disqust;neutral;sad', 'fear;happy', 'fear;happy;neutral', 'fear;neutral', 'happy;neutral', 'happy;neutral;surprise', 'happy;sad', 'happy;surprise', 'neutral;sad', 'neutral;surprise'

<2019년>
'angry;disgust;fear;neutral;surprise', 'angry;fear', 'angry;fear;neutral', 'angry;fear;surprise', 'angry;happy', 'angry;neutral;surprise', 'angry;sad', 'angry;surprise', 'disgust;fear', 'disgust;happy','disgust;neutral;surprise', 'disgust;sad', 'disgust;surprise', 'fear;neutral;surprise', 'fear;sad', 'fear;surprise', 'happy;neutral;sad', 'neutral;sad;surprise', 'sad;surprise'}


In [60]:
encode_dict = {'angry':0, 'disgust':1, 'fear':2,'happy':3,'neutral':4, 'sad':5, 'surprise':6,  
               'neutral;surprise': 450, 'neutral;sad': 460, 'happy;neutral': 340, 
               'angry;neutral': 40, 'disgust;neutral': 140, 'fear;neutral': 240, 
               'happy;surprise': 350, 'angry;happy;neutral': 7340, 'angry;disgust': 10, 
               'happy;neutral;surprise': 3450, 'fear;happy': 230,'fear;happy;neutral': 2340,
               'angry;disgust;neutral': 7140, 'disgust;neutral;sad': 1460, 
               'happy;sad': 360, 'disgust;happy;neutral': 3410, 'angry;fear': 20, 'angry;fear;neutral':7240,
               'angry;fear;surprise': 7250, 'angry;happy': 730, 'angry;neutral;surprise':7450, 
               'angry;sad': 60, 'angry;surprise': 50, 'disgust;fear':120, 'disgust;happy': 130,
               'disgust;neutral;surprise':1450, 'disgust;sad': 160, 'disgust;surprise':150, 
               'fear;neutral;surprise':2450, 'fear;sad': 260, 'fear;surprise':250, 'happy;neutral;sad':3460,
               'neutral;sad;surprise':4560, 'sad;surprise': 560,
               'angry;disgust;fear;neutral;sad': 10000,'angry;disgust;fear;neutral;surprise':20000}

In [61]:
encode_dict = {k: v for k, v in sorted(encode_dict.items(), key=lambda item: item[1])}
print(encode_dict)

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6, 'angry;disgust': 10, 'angry;fear': 20, 'angry;neutral': 40, 'angry;surprise': 50, 'angry;sad': 60, 'disgust;fear': 120, 'disgust;happy': 130, 'disgust;neutral': 140, 'disgust;surprise': 150, 'disgust;sad': 160, 'fear;happy': 230, 'fear;neutral': 240, 'fear;surprise': 250, 'fear;sad': 260, 'happy;neutral': 340, 'happy;surprise': 350, 'happy;sad': 360, 'neutral;surprise': 450, 'neutral;sad': 460, 'sad;surprise': 560, 'angry;happy': 730, 'disgust;neutral;surprise': 1450, 'disgust;neutral;sad': 1460, 'fear;happy;neutral': 2340, 'fear;neutral;surprise': 2450, 'disgust;happy;neutral': 3410, 'happy;neutral;surprise': 3450, 'happy;neutral;sad': 3460, 'neutral;sad;surprise': 4560, 'angry;disgust;neutral': 7140, 'angry;fear;neutral': 7240, 'angry;fear;surprise': 7250, 'angry;happy;neutral': 7340, 'angry;neutral;surprise': 7450, 'angry;disgust;fear;neutral;sad': 10000, 'angry;disgust;fear;neutral;surprise': 2

In [62]:
# encoding Emotion for whole data
# 사전에 실제로 encoding한 끝 수가 마지막 linear layer의 끝자리랑 맞아야 합니다. 아니면 CUDA error: CUBLAS_STATUS_EXECUTION_FAILED가 나는 것 같아요.
# 예를 들어, label이 0~9, 11,13이렇게 12개가 되었어도, 0~13은 14개니까 마지막 레이어에서 14개 unit을 받아야 multiclass classification이 에러없이 진행됩니다!
# 데이터에서 정답 라벨 인코딩: ['neutral', 'happy', 'surprise', 'disgust', 'angry', 'sad', 'fear']
# 이 순서를 지켜서 라벨링을 해야함
decode_dict = {b:i for i, b in encode_dict.items()}
print(encode_dict, '\n', decode_dict)

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6, 'angry;disgust': 10, 'angry;fear': 20, 'angry;neutral': 40, 'angry;surprise': 50, 'angry;sad': 60, 'disgust;fear': 120, 'disgust;happy': 130, 'disgust;neutral': 140, 'disgust;surprise': 150, 'disgust;sad': 160, 'fear;happy': 230, 'fear;neutral': 240, 'fear;surprise': 250, 'fear;sad': 260, 'happy;neutral': 340, 'happy;surprise': 350, 'happy;sad': 360, 'neutral;surprise': 450, 'neutral;sad': 460, 'sad;surprise': 560, 'angry;happy': 730, 'disgust;neutral;surprise': 1450, 'disgust;neutral;sad': 1460, 'fear;happy;neutral': 2340, 'fear;neutral;surprise': 2450, 'disgust;happy;neutral': 3410, 'happy;neutral;surprise': 3450, 'happy;neutral;sad': 3460, 'neutral;sad;surprise': 4560, 'angry;disgust;neutral': 7140, 'angry;fear;neutral': 7240, 'angry;fear;surprise': 7250, 'angry;happy;neutral': 7340, 'angry;neutral;surprise': 7450, 'angry;disgust;fear;neutral;sad': 10000, 'angry;disgust;fear;neutral;surprise': 2

## Annotation Encoding

In [63]:
kemdy19_annot.Emotion = list(kemdy19_annot.Emotion.map(encode_dict))
kemdy20_annot.Emotion = list(kemdy20_annot.Emotion.map(encode_dict))
kemdy19_annot.Emotion[:3], kemdy20_annot.Emotion[:3]


(0    6
 1    2
 2    0
 Name: Emotion, dtype: int64,
 0    4
 1    4
 2    4
 Name: Emotion, dtype: int64)

# EDA, TEMP Padding

In [64]:
import math
def add_padding(pd_series, length = 50):
    if isinstance(pd_series, float):
        if math.isnan(pd_series):
            return np.zeros(10)
    if len(pd_series) < length:
        pd_series = np.concatenate([pd_series, np.zeros(length - len(pd_series))])
        return np.array(pd_series)
    elif len(pd_series) == length:
        return np.array(pd_series)
    elif len(pd_series) > length:
        pd_series = pd_series[:length]
        return np.array(pd_series)

In [65]:
kemdy19_annot.EDA.map(len).describe()

count    9008.000000
mean       22.596470
std        19.693268
min         1.000000
25%         9.000000
50%        17.000000
75%        30.000000
max       314.000000
Name: EDA, dtype: float64

In [66]:
kemdy19_annot['Scaled EDA'] = kemdy19_annot['Scaled EDA'].apply(add_padding)
kemdy20_annot['Scaled EDA'] = kemdy20_annot['Scaled EDA'].apply(add_padding)
kemdy19_annot['Scaled TEMP'] = kemdy19_annot['Scaled TEMP'].apply(add_padding)
kemdy20_annot['Scaled TEMP'] = kemdy20_annot['Scaled TEMP'].apply(add_padding)
# check
kemdy20_annot['Scaled EDA'][10], kemdy19_annot['Scaled EDA'][10], kemdy20_annot['Scaled TEMP'][3],kemdy19_annot['Scaled TEMP'][15]

(array([ 0.33301774,  0.55311825, -0.16032486, -0.10719511,  0.09773054,
         0.10531797,  0.12809209,  0.40890977,  0.23435159, -0.29694591,
        -0.36525051, -0.25140359, -0.46391666, -0.14514408,  0.05978157,
         0.74286905,  0.61383542, -3.12036271,  2.16975526,  1.91929321,
        -0.29693999, -0.19827383, -0.94967184,  0.18880927,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ]),
 array([-0.21123698, -0.18926761, -0.18665215, -0.1861293 , -0.1861293 ,
        -0.18874476, -0.20757542, -0.25517533, -0.29074444, -0.29179054,
        -0.30434459, -0.24105233, -0.24628284, -0.48480603, -0.52822112,
        -0.60860162, -0.77337071, -0.85676544, -0

In [67]:
lengt_wav = 0
lengt_txt = 0
for i,j in zip(kemdy19_emb[0], kemdy19_emb[1]):
    lengt_wav += len(kemdy19_emb[0][i])
    lengt_txt += len(kemdy19_emb[1][j])
    # lengt += len(i)
print(f'session 19 length: wav - {lengt_wav}, txt - {lengt_txt}')

lengt_wav = 0
lengt_txt = 0
for i,j in zip(kemdy20_emb[0], kemdy20_emb[1]):
    lengt_wav += len(kemdy20_emb[0][i])
    lengt_txt += len(kemdy20_emb[1][j])
    
print(f'session 20 length: wav - {lengt_wav}, txt - {lengt_txt}')

session 19 length: wav - 9008, txt - 9008
session 20 length: wav - 12715, txt - 12715


## Session pick 
- test(.2), validation(.2), train(.8)

In [68]:
def choice_and_remove_list(original_list, k = 8):
    removed_new_list = []
    chosen_list = random.sample(original_list, k = k)
    for session in original_list:
        if session in chosen_list:
            pass
        else:
            removed_new_list.append(session) 
    return sorted(removed_new_list), sorted(chosen_list)

In [69]:
# session을 train vs test&val로 나눠줌
session_20_lst = ['Sess0' + str(i+1) if i < 9 else 'Sess' + str(i+1) for i in range(40)]
session_20_lst.remove('Sess12')
session_20_lst.remove('Sess17')
sessions_20_train_lst, sessions_20_test_lst = choice_and_remove_list(session_20_lst, k = 8)
sessions_20_train_lst, sessions_20_val_lst = choice_and_remove_list(sessions_20_train_lst, k = 8)
print(sessions_20_train_lst, sessions_20_test_lst, sessions_20_val_lst, sep = '\n')

session_19_lst = ['Sess0' + str(i+1) if i < 9 else 'Sess' + str(i+1) for i in range(20)]
sessions_19_train_lst, sessions_19_test_lst = choice_and_remove_list(session_19_lst, k = 0)
sessions_19_train_lst, sessions_19_val_lst = choice_and_remove_list(sessions_19_train_lst, k = 0)
print('\n', sessions_19_train_lst,sessions_19_test_lst, sessions_19_val_lst, sep = '\n')

['Sess01', 'Sess02', 'Sess03', 'Sess07', 'Sess10', 'Sess11', 'Sess13', 'Sess15', 'Sess16', 'Sess19', 'Sess22', 'Sess23', 'Sess24', 'Sess25', 'Sess26', 'Sess30', 'Sess31', 'Sess32', 'Sess33', 'Sess34', 'Sess37', 'Sess38']
['Sess05', 'Sess06', 'Sess18', 'Sess21', 'Sess28', 'Sess29', 'Sess36', 'Sess40']
['Sess04', 'Sess08', 'Sess09', 'Sess14', 'Sess20', 'Sess27', 'Sess35', 'Sess39']


['Sess01', 'Sess02', 'Sess03', 'Sess04', 'Sess05', 'Sess06', 'Sess07', 'Sess08', 'Sess09', 'Sess10', 'Sess11', 'Sess12', 'Sess13', 'Sess14', 'Sess15', 'Sess16', 'Sess17', 'Sess18', 'Sess19', 'Sess20']
[]
[]


## data reconstruct
- dict - session - txt, wav, segment_id

In [70]:
kemdy20_emb_new = {}
for session in kemdy20_emb[0].keys():
    kemdy20_emb_new[session] = {}
    kemdy20_emb_new[session]['wav'] = kemdy20_emb[0][session]
    kemdy20_emb_new[session]['txt'] = kemdy20_emb[1][session]
    kemdy20_emb_new[session]['segment_id'] = kemdy20_annot['Segment ID'][kemdy20_annot['Segment ID'].str.startswith(session)]
    # print(session, len(kemdy20_emb_new[session]['wav']), len(kemdy20_emb_new[session]['txt']), len(kemdy20_emb_new[session]['segment_id']))

# print('\n')
kemdy19_emb_new = {}
for session in kemdy19_emb[0].keys():
    kemdy19_emb_new[session] = {}
    kemdy19_emb_new[session]['wav'] = kemdy19_emb[0][session]
    kemdy19_emb_new[session]['txt'] = kemdy19_emb[1][session]
    kemdy19_emb_new[session]['segment_id'] = kemdy19_annot['Segment ID'][kemdy19_annot['Segment ID'].str.startswith(session)]
    # print(session, len(kemdy19_emb_new[session]['wav']), len(kemdy19_emb_new[session]['txt']), len(kemdy19_emb_new[session]['segment_id']))


In [71]:
# 나눠준 세션을 emb(wav, text순), annot set에 적용
def get_data_by_session(data, session_lst):
    if isinstance(data, pd.DataFrame):
        print('dataframe')
        for idx, session in enumerate(session_lst):
            if idx == 0:
                dataframe = data[data['Segment ID'].str.startswith(session)]
            else:
                dataframe = pd.concat([dataframe, data[data['Segment ID'].str.startswith(session)]])
        return dataframe
    
    elif isinstance(data, dict):
        print('dict')
        emb_data = {}
        emb_data['wav'] = []
        emb_data['txt'] = []
        emb_data['segment_id'] = []
        for session in session_lst:
            emb_data['wav'].extend(data[session]['wav'])
            emb_data['txt'].extend(data[session]['txt'])
            emb_data['segment_id'].extend(data[session]['segment_id'])            
        return emb_data

In [72]:
kemdy19_annot_train = get_data_by_session(kemdy19_annot, sessions_19_train_lst)
# kemdy19_annot_test = get_data_by_session(kemdy19_annot, sessions_19_test_lst)
# kemdy19_annot_val = get_data_by_session(kemdy19_annot, sessions_19_val_lst)

kemdy20_annot_train = get_data_by_session(kemdy20_annot, sessions_20_train_lst)
kemdy20_annot_test = get_data_by_session(kemdy20_annot, sessions_20_test_lst)
kemdy20_annot_val = get_data_by_session(kemdy20_annot, sessions_20_val_lst)

dataframe
dataframe
dataframe
dataframe


In [73]:
# session 정보가 없어집니다. 
kemdy19_emb_train = get_data_by_session(kemdy19_emb_new, sessions_19_train_lst)
# kemdy19_emb_test = get_data_by_session(kemdy19_emb_new, sessions_19_test_lst)
# kemdy19_emb_val = get_data_by_session(kemdy19_emb_new, sessions_19_val_lst)

kemdy20_emb_train = get_data_by_session(kemdy20_emb_new, sessions_20_train_lst)
kemdy20_emb_test = get_data_by_session(kemdy20_emb_new, sessions_20_test_lst)
kemdy20_emb_val = get_data_by_session(kemdy20_emb_new, sessions_20_val_lst)

dict
dict
dict
dict


In [74]:
# # kemdy 19 - 9008이 정상
# print(len(kemdy19_emb_train['wav']), len(kemdy19_emb_train['txt']), len(kemdy19_emb_train['segment_id']))
# print(len(kemdy19_emb_test['wav']), len(kemdy19_emb_test['txt']), len(kemdy19_emb_test['segment_id']))
# print(len(kemdy19_emb_val['wav']), len(kemdy19_emb_val['txt']), len(kemdy19_emb_val['segment_id']))
# # print(len(kemdy19_emb_train['wav']) + len(kemdy19_emb_test['wav']) + len(kemdy19_emb_val['wav']))

In [75]:
# kemdy 20 - 12715가 정상
print(len(kemdy20_emb_train['wav']), len(kemdy20_emb_train['txt']), len(kemdy20_emb_train['segment_id']))
print(len(kemdy20_emb_test['wav']), len(kemdy20_emb_test['txt']), len(kemdy20_emb_test['segment_id']))
print(len(kemdy20_emb_val['wav']), len(kemdy20_emb_val['txt']), len(kemdy20_emb_val['segment_id']))
print(len(kemdy20_emb_train['wav']) + len(kemdy20_emb_test['wav'])+ len(kemdy20_emb_val['wav']))

7514 7514 7514
2442 2442 2442
2759 2759 2759
12715


In [76]:
# print(kemdy19_emb_train.keys(), kemdy19_emb_test.keys(), kemdy19_emb_val.keys(),
#       kemdy20_emb_train.keys(),kemdy20_emb_test.keys(), kemdy20_emb_val.keys(),sep = '\n')

## Train Data neutral pick하기

In [77]:
# 각 데이터 셋에서 몇개 뽑아야 되는지 계산 neutral: 4
target_neutral_num = 5000

target_neutral_num_19 = 0
# target_neutral_num_19 = int(target_neutral_num / (Counter(kemdy19_annot['Emotion'])[4] + Counter(kemdy20_annot['Emotion'])[4]) * Counter(kemdy19_annot['Emotion'])[4])
target_neutral_num_20 = target_neutral_num - target_neutral_num_19
print(f'kemdy19에서 {target_neutral_num_19}개, kemdy20에서 {target_neutral_num_20}개 추출')

kemdy19에서 0개, kemdy20에서 5000개 추출


In [78]:
# 각 train dataset에서 뽑아야 되는 갯수만큼 랜덤으로 뽑아옴
kemdy19_annot_train_not_neut = kemdy19_annot_train[kemdy19_annot_train['Emotion'] != 4]
kemdy20_annot_train_not_neut = kemdy20_annot_train[kemdy20_annot_train['Emotion'] != 4]

kemdy19_annot_train_neut = kemdy19_annot_train[kemdy19_annot_train['Emotion'] == 4].sample(target_neutral_num_19)
kemdy20_annot_train_neut = kemdy20_annot_train[kemdy20_annot_train['Emotion'] == 4].sample(target_neutral_num_20)
len(kemdy19_annot_train_not_neut) + len(kemdy19_annot_train_neut) + len(kemdy20_annot_train_not_neut) + len(kemdy20_annot_train_neut)

11925

## embedding, Test and validation dataset 합치기

In [79]:
emb_test_final = {}
emb_test_final['wav'] = []
emb_test_final['txt'] = []
emb_test_final['segment_id'] = []

# emb_test_final['wav'] = kemdy19_emb_test['wav']
emb_test_final['wav'].extend(kemdy20_emb_test['wav'])

# emb_test_final['txt'] = kemdy19_emb_test['txt']
emb_test_final['txt'].extend(kemdy20_emb_test['txt'])


# emb_test_final['segment_id'] = kemdy19_emb_test['segment_id']
emb_test_final['segment_id'].extend(kemdy20_emb_test['segment_id'])


print(len(emb_test_final['wav']), len(emb_test_final['txt']), len(emb_test_final['segment_id']))

2442 2442 2442


In [80]:
emb_val_final = {}
emb_val_final['wav'] = []
emb_val_final['txt'] = []
emb_val_final['segment_id'] = []

# emb_val_final['wav'] = kemdy19_emb_val['wav']
emb_val_final['wav'].extend(kemdy20_emb_val['wav'])

# emb_val_final['txt'] = kemdy19_emb_val['txt']
emb_val_final['txt'].extend(kemdy20_emb_val['txt'])

# emb_val_final['segment_id'] = kemdy19_emb_val['segment_id']
emb_val_final['segment_id'].extend(kemdy20_emb_val['segment_id'])

print(len(emb_val_final['wav']), len(emb_val_final['txt']), len(emb_val_final['segment_id']))

2759 2759 2759


In [81]:

annot_train_final = pd.concat([kemdy19_annot_train_neut, kemdy20_annot_train_neut, kemdy19_annot_train_not_neut, kemdy20_annot_train_not_neut])
# annot_test_final = pd.concat([kemdy19_annot_test, kemdy20_annot_test])
# annot_val_final = pd.concat([kemdy19_annot_val, kemdy20_annot_val])

annot_test_final = pd.concat([kemdy20_annot_test])
annot_val_final = pd.concat([kemdy20_annot_val])


annot_train_final.reset_index(drop=True, inplace=True)
annot_test_final.reset_index(drop=True, inplace=True)
annot_val_final.reset_index(drop=True, inplace=True)

print(len(annot_train_final), len(annot_test_final), len(annot_val_final))

11925 2442 2759


In [82]:
# train dataset neutral 4000개로 랜덤 뽑은 것 생성
emb_train_final = {}
emb_train_final['wav'] = []
emb_train_final['txt'] = []
emb_train_final['segment_id'] = []
for segment_annot_id in kemdy19_annot_train_neut['Segment ID']:
    for wav, txt, segment_emb_id in zip(kemdy19_emb_train['wav'], kemdy19_emb_train['txt'], kemdy19_emb_train['segment_id']):
        if segment_annot_id == segment_emb_id:
            emb_train_final['wav'].append(wav)
            emb_train_final['txt'].append(txt)
            emb_train_final['segment_id'].append(segment_emb_id)
            
for segment_annot_id in kemdy19_annot_train_not_neut['Segment ID']:
    for wav, txt, segment_emb_id in zip(kemdy19_emb_train['wav'], kemdy19_emb_train['txt'], kemdy19_emb_train['segment_id']):
        if segment_annot_id == segment_emb_id:
            emb_train_final['wav'].append(wav)
            emb_train_final['txt'].append(txt)
            emb_train_final['segment_id'].append(segment_emb_id)
        

for segment_annot_id in kemdy20_annot_train_neut['Segment ID']:
    for wav, txt, segment_emb_id in zip(kemdy20_emb_train['wav'], kemdy20_emb_train['txt'], kemdy20_emb_train['segment_id']):
        if segment_emb_id == segment_annot_id:
            emb_train_final['wav'].append(wav)
            emb_train_final['txt'].append(txt)
            emb_train_final['segment_id'].append(segment_emb_id)
        
for segment_annot_id in kemdy20_annot_train_not_neut['Segment ID']:
    for wav, txt, segment_emb_id in zip(kemdy20_emb_train['wav'], kemdy20_emb_train['txt'], kemdy20_emb_train['segment_id']):
        if segment_annot_id == segment_emb_id:
            emb_train_final['wav'].append(wav)
            emb_train_final['txt'].append(txt)
            emb_train_final['segment_id'].append(segment_emb_id)
            
print(len(emb_train_final['wav']), len(emb_train_final['txt']), len(emb_train_final['segment_id']))

11925 11925 11925


# torch dataset 만들기
- 참고: https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html

In [83]:
torch.set_default_dtype(torch.float32)
print(torch.get_default_dtype())

torch.float32


In [84]:
class EtriDataset(Dataset):
    def __init__(self, file_names, 
                 text_embeddings, 
                 wav_embeddings, 
                 Temp,
                 EDA,
                 Emotion,
                 Emotion_vec, 
                 Arousal, 
                 Valence):
        self.file_names = file_names
        self.text_embeddings = text_embeddings
        self.wav_embeddings = wav_embeddings
        self.temp = Temp
        self.eda = EDA
        self.label_emotion = Emotion
        self.label_emotion_vec = Emotion_vec
        self.label_arousal = Arousal
        self.label_valence = Valence
        
        
    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        text_embeddings = self.text_embeddings[idx]
        wav_embeddings = self.wav_embeddings[idx]
        temp = self.temp[idx]
        eda = self.eda[idx]
        label_emotion = self.label_emotion[idx]
        label_emotion_ext = self.label_emotion_vec[idx]
        label_arousal = self.label_arousal[idx]
        label_valence = self.label_valence[idx]
        return text_embeddings, wav_embeddings, temp, eda, label_emotion, label_emotion_ext, label_arousal, label_valence

In [85]:
annot_train_final.columns

Index(['Segment ID', 'Emotion', 'Valence', 'Arousal', 'emotion_vector',
       'valence_vector', 'arousal_vector', 'EDA', 'TEMP', 'EDA length',
       'TEMP length', 'Scaled EDA', 'Scaled TEMP'],
      dtype='object')

In [86]:
annot_train_final['Scaled EDA']

0        [0.0672257114501823, -1.1190632255018322, 0.46...
1        [0.022837937219205698, -0.8803725498531865, 2....
2        [0.6005758414230128, 0.14718366123743523, -0.4...
3        [-0.04966521073827487, -0.04966521073827487, -...
4        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                               ...                        
11920    [1.0248824293032237, -0.8716879365334415, -0.1...
11921    [0.9738374515615607, -0.011967660642870578, -0...
11922    [-0.011967660642870578, -0.011967660642870578,...
11923    [-0.011967660642870578, -0.011967660642870578,...
11924    [-0.16042779153088987, -0.16061282278609879, 0...
Name: Scaled EDA, Length: 11925, dtype: object

In [87]:
# data load 및 나누기: https://076923.github.io/posts/Python-pytorch-11/

# annot_train_final, annot_test_final, annot_val_final
# emb_train_final, emb_test_final, emb_val_final

# session을 통합시킨 데이터 셋을 만들었을 때
dataset_train = EtriDataset(file_names = annot_train_final['Segment ID'],
                      text_embeddings = torch.stack(emb_train_final['txt']),
                      wav_embeddings = torch.stack(emb_train_final['wav']),
                      Emotion = annot_train_final['Emotion'],
                      Arousal = annot_train_final['Arousal'],
                      Valence = annot_train_final['Valence'],
                      EDA = torch.Tensor(annot_train_final['Scaled EDA']), 
                      Temp = torch.Tensor(annot_train_final['Scaled TEMP']), 
                      Emotion_vec = torch.Tensor(annot_train_final['emotion_vector'])) 


dataset_test = EtriDataset(file_names = annot_test_final['Segment ID'],
                      text_embeddings = torch.stack(emb_test_final['txt']),
                      wav_embeddings = torch.stack(emb_test_final['wav']),
                      Emotion = annot_test_final['Emotion'],
                      Arousal = annot_test_final['Arousal'],
                      Valence = annot_test_final['Valence'],
                      EDA = torch.Tensor(annot_test_final['Scaled EDA']), 
                      Temp = torch.Tensor(annot_test_final['Scaled TEMP']), 
                      Emotion_vec = torch.Tensor(annot_test_final['emotion_vector']))

dataset_val = EtriDataset(file_names = annot_val_final['Segment ID'],
                      text_embeddings = torch.stack(emb_val_final['txt']),
                      wav_embeddings = torch.stack(emb_val_final['wav']),
                      Emotion = annot_val_final['Emotion'],
                      Arousal = annot_val_final['Arousal'],
                      Valence = annot_val_final['Valence'],
                      EDA = torch.Tensor(annot_val_final['Scaled EDA']), 
                      Temp = torch.Tensor(annot_val_final['Scaled TEMP']), 
                      Emotion_vec = torch.Tensor(annot_val_final['emotion_vector'])) 


In [88]:
print(f"Training Data Size : {len(dataset_train)}")
print(f"Validation Data Size : {len(dataset_val)}")
print(f"Testing Data Size : {len(dataset_test)}")

Training Data Size : 11925
Validation Data Size : 2759
Testing Data Size : 2442


In [89]:
train_dataloader = DataLoader(dataset_train, batch_size=512, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(dataset_val, batch_size=128, shuffle=True, drop_last=True)
test_dataloader = DataLoader(dataset_test, batch_size=128, shuffle=True, drop_last=True)

# NetWork 만들기

In [90]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [91]:
class MLPNetwork_pre(nn.Module):
    def __init__(self, input_length, input_width):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_length*input_width, 768)
        self.gelu1 = nn.GELU()
        self.bn1 = nn.BatchNorm1d(768)
        self.fc2 = nn.Linear(768, 512)
        self.gelu2 = nn.GELU()
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 32)
        self.gelu3 = nn.GELU()
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.gelu1(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.gelu2(x)
        x = self.bn2(x)
        x = self.fc3(x)
        output = self.gelu3(x)
        return output
    
class ConvNetwork_pre(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels = 1, out_channels= 1, kernel_size = 16)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels = 1, out_channels = 1, kernel_size = 16)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = torch.transpose(x, 1,2)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        output = self.relu2(x)
        return output

class ConvNetwork_middle(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=1, kernel_size=11)
        self.relu1 = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        output = self.relu1(x)
        return output.squeeze()

class ConvNetwork_final(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv2d_1 = nn.Conv2d(in_channels = 10, out_channels = 64, kernel_size=2)
        self.leakyrelu_1 = nn.LeakyReLU()
        self.maxpool2d_1 = nn.MaxPool2d(2)
        self.conv2d_2 = nn.Conv2d(in_channels = 64, out_channels = 32, kernel_size=2)
        self.leakyrelu_2 = nn.LeakyReLU()
        self.maxpool2d_2 = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(1568, 64)
        self.leakyrelu_3 = nn.LeakyReLU()
        self.batchnorm = nn.BatchNorm1d(64)
        self.drop = nn.Dropout(p=0.25)
        self.fc2 = nn.Linear(64, 7)

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.leakyrelu_1(x)
        x = self.maxpool2d_1(x)
        x = self.conv2d_2(x)
        x = self.leakyrelu_2(x)
        x = self.maxpool2d_2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.leakyrelu_3(x)
        x = self.batchnorm(x)
        x = self.drop(x)
        output = self.fc2(x)
        return output
        



In [92]:
class TensorFusionMixer(nn.Module):
    def __init__(self, ModelA, ModelB, ModelC, ModelD, ModelE, ModelF):
        super().__init__()
        self.ModelA = ModelA
        self.ModelB = ModelB
        self.ModelC = ModelC
        self.ModelD = ModelD
        self.ModelE = ModelE
        self.Model_cnn_final = ModelF
        # self.softmax = nn.Softmax(dim=1)

    def tensor_fusion(self, batch_arr1, batch_arr2, batch_arr3):
        fusion_matrix_lst = []
        for i, (arr1, arr2, arr3) in enumerate(zip(batch_arr1, batch_arr2, batch_arr3)):
            # arr1 = arr1.unsqueeze(-1).unsqueeze(-1)
            # arr2 = arr2.unsqueeze(0).unsqueeze(-1)
            # arr3 = arr3.squeeze().unsqueeze(0).unsqueeze(0)

            arr1 = arr1.unsqueeze(0).unsqueeze(0)
            arr2 = arr2.unsqueeze(0).unsqueeze(-1)
            arr3 = arr3.unsqueeze(-1).unsqueeze(-1)

            # outer_matrix = torch.einsum('i,j,kp->ijk', arr1, arr2, arr3)
            kron_matrix = torch.kron(arr3, torch.kron(arr2, arr1,))
            l, w, d = kron_matrix.shape

            kron_matrix = kron_matrix.view(-1, l, w, d)
            fusion_matrix_lst.append(kron_matrix)

        fusion_matrix = torch.concat(fusion_matrix_lst)
        # fusion_matrix = fusion_matrix.unsqueeze(-1)

        return fusion_matrix
    
    def forward(self, x1, x2, x3, x4):
            x1 = self.ModelA(x1)
            x2 = self.ModelB(x2)
            x3 = self.ModelC(x3)
            x4 = self.ModelD(x4)
            x5 = torch.concat([x3, x4], dim=1)
            x5 = self.ModelE(x5)
            # print(x1.shape, x2.shape, x5.shape)
            fusion_matrix = self.tensor_fusion(x1, x2, x5)
            # x5 = torch.cat([x3,x4], dim=0)
            output = self.Model_cnn_final(fusion_matrix) 
            return output

In [93]:
# txt_input_length, txt_input_width = raw_dataset[session]['textembeddings'][0].shape | 마지막엔 지울 것
# , wav_input_length, wav_input_width = raw_dataset[session]['wav_embeddings'][0].shape
txt_input_length, txt_input_width = torch.Tensor(emb_train_final['txt'][0]).shape
wav_input_length, wav_input_width = torch.Tensor(emb_train_final['wav'][0]).shape
# temp_input_length = annot_train_final['Scaled EDA'][0].shape[0]
# eda_input_length = annot_train_final['Scaled TEMP'][0].shape[0]

# tf_mixer에 들어갈 wav mlp, txt mlp 선언
model_mlp_txt = MLPNetwork_pre(txt_input_length,txt_input_width).to(device)
model_mlp_wav = MLPNetwork_pre(wav_input_length,wav_input_width).to(device)

model_conv_temp = ConvNetwork_pre().to(device)
model_conv_eda = ConvNetwork_pre().to(device)

model_conv_middle = ConvNetwork_middle().to(device)

model_cnn_final = ConvNetwork_final().to(device)

# 최종 모델 선언
model_tf_cnn_mixer = TensorFusionMixer(ModelA = model_mlp_txt, 
                                   ModelB = model_mlp_wav,
                                   ModelC = model_conv_temp,
                                   ModelD = model_conv_eda,
                                   ModelE = model_conv_middle,
                                   ModelF = model_cnn_final).to(device)

# model 병렬 학습 처리
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model_mlp_txt = nn.DataParallel(model_mlp_txt).to(device)
    model_mlp_wav = nn.DataParallel(model_mlp_wav).to(device)
    model_conv_temp = nn.DataParallel(model_conv_temp).to(device)
    model_conv_eda = nn.DataParallel(model_conv_eda).to(device)
    model_tf_cnn_mixer = nn.DataParallel(model_tf_cnn_mixer).to(device)

print(model_tf_cnn_mixer)

Let's use 4 GPUs!
DataParallel(
  (module): TensorFusionMixer(
    (ModelA): MLPNetwork_pre(
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (fc1): Linear(in_features=61440, out_features=768, bias=True)
      (gelu1): GELU(approximate='none')
      (bn1): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Linear(in_features=768, out_features=512, bias=True)
      (gelu2): GELU(approximate='none')
      (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc3): Linear(in_features=512, out_features=32, bias=True)
      (gelu3): GELU(approximate='none')
    )
    (ModelB): MLPNetwork_pre(
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (fc1): Linear(in_features=152576, out_features=768, bias=True)
      (gelu1): GELU(approximate='none')
      (bn1): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Linear(in_features=768, out_features=512, bias=T

# 학습을 위한 train, test method 만들기

In [94]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)    
    # data 순서: text_embeddings, wav_embeddings, temp, eda, label_emotion, label_emotion_ext, label_arousal, label_valence
    for batch, (X_txt, X_wav, X_temp, X_eda, 
                    label_emotion, label_emotion_vec, label_arousal, label_valence) in enumerate(dataloader): 
        y = label_emotion_vec # 라벨을 변경하고자 하면 이 변수만 바꿔주면 나머지는 y로 적용
        # 예측 오류 계산 
        X_txt, X_wav, X_temp, X_eda, y= X_txt.to(device), X_wav.to(device), X_temp.to(device), X_eda.to(device),y.type(torch.float32).to(device)
        
        X_temp = X_temp.unsqueeze(dim=-1)
        X_eda = X_eda.unsqueeze(dim=-1)
        
        pred = model(X_txt, X_wav, X_temp, X_eda)
        y = F.softmax(y, dim = 1)
        
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.mean().backward() # weighted MSE를 사용할 경우 중간에 sum() or mean()을 넣어줌
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.mean().mean().item(), batch * len(X_txt) # weighted MSE를 사용할 경우 중간에 sum() or mean()을 넣어줌
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        

In [95]:
def cal_multiple_class(probs, threshold = 0.1):
    values = probs.topk(2)
    pred_list = []
    # print(values)
    diffs = abs(torch.diff(values.values))
    for idx,diff in zip(values.indices, diffs):
        if diff <= threshold:
            sorted, idx = torch.sort(idx)
            if sorted[0] == 0:
                pred_list.append(100*7 + 10*sorted[1].item())
            else:
                pred_list.append(100*sorted[0].item() + 10*sorted[1].item())
        else:
            pred_list.append(idx[0].item())
    return torch.Tensor(pred_list).to(device), probs.argmax(1)

In [96]:
from copy import deepcopy


def test(dataloader, model, loss_fn, mode = 'test'):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    f1 = F1Score(task= 'multiclass', num_classes=37).to(device)   # 바로 multiclassification할 경우
    preds = []
    targets = []
    
    with torch.no_grad():
        # data 순서: text_embeddings, wav_embeddings, temp, eda, label_emotion, label_emotion_ext, label_arousal, label_valence
        for batch, (X_txt, X_wav, X_temp, X_eda, 
                        label_emotion, label_emotion_vec, label_arousal, label_valence) in enumerate(dataloader): 
            y = label_emotion_vec # 라벨을 변경하고자 하면 이 변수만 바꿔주면 나머지는 y로 적용
            # 예측 오류 계산
            X_txt, X_wav, X_temp, X_eda, y= X_txt.to(device), X_wav.to(device), X_temp.to(device), X_eda.to(device),y.type(torch.float32).to(device)
            
            X_temp = X_temp.unsqueeze(dim=-1)
            X_eda = X_eda.unsqueeze(dim=-1)
            
            pred = model(X_txt, X_wav, X_temp, X_eda)
            pred_for_acc, _ = cal_multiple_class(pred)
            
            preds.append(pred_for_acc)
            y = F.softmax(y, dim = 1)
            # preds.append(pred.argmax(1)) # multi regression후 classification으로 변환할 경우

            targets.append(label_emotion) # classification을 할 경우 언제나 사용
            # print('예측라벨분포:',pred[:2], '정답라벨 분포:', label_emotion_ext[:2], '예측정답:', pred.argmax(1)[:2],'정답:', label_emotion[:2])
            # print('예측:', pred.argmax(1).tolist()[:2],'\n', '정답:', label_emotion.tolist()[:2])
            # https://discuss.pytorch.org/t/loss-backward-raises-error-grad-can-be-implicitly-created-only-for-scalar-outputs/12152/6
            test_loss += loss_fn(pred, y).mean().item()# weighted MSE를 사용할 경우 중간에 sum() or mean()을 넣어줌 
            
            correct += (pred.argmax(1) == label_emotion.to(device)).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    f1_score = f1(torch.cat(preds).to(device), torch.cat(targets).to(device))
    accuracy = (100*correct)
    
    if mode == 'test':
        print(torch.cat(preds), torch.cat(preds).shape)
        print("f1 score: ", f1_score)
        print(f"Test Error: Accuracy: {(accuracy):>0.1f}%, Avg loss: {test_loss:>8f}\n")
    elif mode == 'val':
        print(f"Validation Error: Accuracy: {(accuracy):>0.1f}%, Avg val loss: {test_loss:>8f} \n")
    
    return f1_score, accuracy, test_loss

# 학습시키기

In [97]:
# weigted loss for imbalance data: https://naadispeaks.wordpress.com/2021/07/31/handling-imbalanced-classes-with-weighted-loss-in-pytorch/
# weight 계산
single_emotion = [0,1,2,3,4,5,6]
total_obs = 0
for i in single_emotion:
    total_obs += Counter(annot_train_final['Emotion'])[i]
print('total obs: ', total_obs)

weight_for_class = []
print(Counter(annot_train_final['Emotion']))
for key, value in sorted(Counter(annot_train_final['Emotion']).items()):
    if key in single_emotion:
        print(f'{key} is in single emotion')
        if key == 'neutral':
            weight_for_class.append(1 - (1440/total_obs))
        else:
            weight_for_class.append(1 - (value/total_obs))
weight_for_class = torch.Tensor(weight_for_class).type(torch.float16)
weight_for_class

total obs:  10975
Counter({4: 5000, 3: 1856, 0: 1603, 6: 1008, 5: 731, 1: 394, 2: 383, 340: 345, 40: 128, 450: 105, 460: 90, 140: 77, 10: 41, 350: 31, 250: 23, 240: 21, 50: 13, 60: 13, 20: 9, 7140: 6, 560: 6, 7340: 4, 260: 3, 150: 3, 730: 3, 7240: 3, 120: 3, 1450: 2, 230: 2, 4560: 2, 3450: 2, 2340: 2, 1460: 2, 360: 2, 7250: 1, 7450: 1, 2450: 1, 3460: 1, 130: 1, 20000: 1, 160: 1, 10000: 1, 3410: 1})
0 is in single emotion
1 is in single emotion
2 is in single emotion
3 is in single emotion
4 is in single emotion
5 is in single emotion
6 is in single emotion


tensor([0.8540, 0.9639, 0.9653, 0.8311, 0.5444, 0.9336, 0.9082],
       dtype=torch.float16)

In [98]:
class weighted_MSELoss(nn.Module):
    def __init__(self, weight):
        super().__init__()
        self.weight = weight.to(device)
    def forward(self,inputs,targets):
        return ((inputs - targets)**2) * self.weight

In [99]:
# # 지난 저장한 모델이 있다면
# PATH = './data/test_model.pkl'
# model_tf_mixer = torch.load(PATH)

In [100]:
# loss_fn = nn.CrossEntropyLoss(weight=weight_for_class).to(device)
# loss_fn = nn.CrossEntropyLoss().to(device) # weigth를 주기위해 위의 loss로 임시 변경
loss_fn = weighted_MSELoss(weight = weight_for_class).to(device) # multi target regression(감정별로 count 한 타겟)
# loss_fn = nn.MSELoss().to(device) 

In [101]:
lr = 1e-4

# optimizer = optim.SGD(model_tf_mixer.parameters(), lr=lr) # classification
optimizer = optim.Adagrad(model_tf_cnn_mixer.parameters(), lr=lr) # regression

## start training mlp fusion mixer

In [115]:
import wandb

!wandb login --relogin "9bccad4ccb69d37aa82b7b6b8e2e2d3c54f79ab1"
wandb.login()

wandb: Appending key for api.wandb.ai to your netrc file: /home/arplab/.netrc


True

In [119]:
#wandb
epochs = 50

wandb.init(
    # set the wandb project where this run will be logged
    project="ETRI-lou-multiclassification",
    name = f'Experiment Name',
    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "CNN Tensor Fusion Mixer",
    "dataset": "ETRI Kemdy20",
    "epochs": epochs,
    "Optimizer": optimizer.__class__.__name__,
    "Loss": loss_fn.__class__.__name__
    }
)

accuracy,▁
loss,▁
accuracy,0.57992
loss,0.07625


In [120]:
# Set the Training Parameters
from copy import deepcopy
loss_list = []
acc_list = []
best_acc = 0
best_f1 = 0
best_acc_model = None 
best_f1_model = None


for epoch in range(epochs):
    print(f"---------------Epoch {epoch+1}----------------")
    train(train_dataloader, model_tf_cnn_mixer, loss_fn, optimizer)
    f1_score, accuracy, loss = test(validation_dataloader, model_tf_cnn_mixer, loss_fn, mode = 'val')
    if accuracy > best_acc:
        best_acc = accuracy
        best_acc_model = deepcopy(model_tf_cnn_mixer.state_dict())
        print('best_acc:', best_acc)
    if f1_score > best_f1:
        best_f1 = f1_score
        best_f1_model = deepcopy(model_tf_cnn_mixer.state_dict())
        print('best_f1:', best_f1)
    loss_list.append(loss)
    acc_list.append(accuracy)
    wandb.log({'accuracy': accuracy, 'loss': loss})
wandb.finish()
print("Done!")

---------------Epoch 1----------------
loss: 0.084427  [    0/11925]
Validation Error: Accuracy: 81.2%, Avg val loss: 0.038802 

best_acc: 81.15259151866618
best_f1: tensor(0.8103, device='cuda:0')
---------------Epoch 2----------------
loss: 0.062894  [    0/11925]
Validation Error: Accuracy: 79.5%, Avg val loss: 0.038152 

---------------Epoch 3----------------
loss: 0.062250  [    0/11925]
Validation Error: Accuracy: 76.0%, Avg val loss: 0.044005 

---------------Epoch 4----------------
loss: 0.067057  [    0/11925]
Validation Error: Accuracy: 76.8%, Avg val loss: 0.040807 

---------------Epoch 5----------------
loss: 0.060425  [    0/11925]
Validation Error: Accuracy: 63.1%, Avg val loss: 0.051768 

---------------Epoch 6----------------
loss: 0.063029  [    0/11925]
Validation Error: Accuracy: 79.6%, Avg val loss: 0.035154 

---------------Epoch 7----------------
loss: 0.059550  [    0/11925]
Validation Error: Accuracy: 69.2%, Avg val loss: 0.044627 

---------------Epoch 8------

KeyboardInterrupt: 

In [122]:
# 실험을 위해 모델 저장
PATH = '../model/data/model_multilabel_CNNf1_8103.pkl'
torch.save(best_f1_model, PATH)

In [59]:
test(test_dataloader, model_tf_cnn_mixer, loss_fn, mode = 'test')

tensor([0, 4, 4,  ..., 4, 0, 4], device='cuda:0') torch.Size([4480])
f1 score:  tensor(0.4583, device='cuda:0')
Test Error: Accuracy: 45.4%, Avg loss: 0.065306



In [52]:
for batch, (X_txt, X_wav, X_temp, X_eda, 
                        label_emotion, label_emotion_ext, label_arousal, label_valence) in list(enumerate(test_dataloader))[:4]:
    print(X_txt,X_wav,label_emotion)

tensor([[[-0.4593, -0.0085, -1.1601,  ..., -0.7082,  1.1274, -0.3243],
         [-0.5511,  0.0475, -1.1391,  ..., -0.6807,  1.1464, -0.2989],
         [-0.4926,  0.0148, -1.1642,  ..., -0.6688,  1.1545, -0.3236],
         ...,
         [-0.4815,  0.0169, -1.2180,  ..., -0.6512,  0.9577, -0.3519],
         [-0.4815,  0.0169, -1.2180,  ..., -0.6512,  0.9577, -0.3519],
         [-0.4815,  0.0169, -1.2180,  ..., -0.6512,  0.9577, -0.3519]],

        [[-0.4723,  0.0884, -1.1636,  ..., -0.7154,  1.0938, -0.2687],
         [-0.5448,  0.1459, -1.0904,  ..., -0.6576,  1.0684, -0.1930],
         [-0.5538,  0.0875, -1.1415,  ..., -0.7035,  1.1440, -0.2562],
         ...,
         [-0.4962,  0.1042, -1.2488,  ..., -0.6564,  0.9447, -0.2766],
         [-0.4962,  0.1042, -1.2488,  ..., -0.6564,  0.9447, -0.2766],
         [-0.4962,  0.1042, -1.2488,  ..., -0.6564,  0.9447, -0.2766]],

        [[-0.5212, -0.0179, -1.0805,  ..., -0.7620,  1.1764, -0.2533],
         [-0.5802,  0.0212, -1.0472,  ..., -0

In [56]:
X_temp_new = X_temp.unsqueeze(dim=-1)
X_eda_new = X_eda.unsqueeze(dim=-1)

probs = model_tf_cnn_mixer(X_txt.to(device), X_wav.to(device), X_temp_new.to(device), X_eda_new.to(device))

print(probs.argmax(1))


values = probs.topk(2)
pred_list = []
print(values)
diffs = abs(torch.diff(values.values))
for idx,diff in zip(values.indices, diffs):
    if diff <= 0.1:
        sorted_value, idx = torch.sort(idx)
        if sorted_value[0] == 0:
            pred_list.append(100*7 + 10*sorted_value[1].item())
        else:
            pred_list.append(100*sorted_value[0].item() + 10*sorted_value[1].item())
    else:
        pred_list.append(idx[0].item())
print(pred_list)



tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6], device='cuda:0')
torch.return_types.topk(
values=tensor([[0.1579, 0.1481],
        [0.1575, 0.1474],
        [0.1567, 0.1479],
        [0.1591, 0.1482],
        [0.1564, 0.1472],
        [0.1607, 0.1521],
        [0.1564, 0.1482],
        [0.1627, 0.1531],
        [0.1582, 0.1491],
        [0.1610, 0.1504],
        [0.1581, 0.1471],
        [0.1606, 0.1497],
        [0.1583, 0.1478],
        [0.1581, 0.1475],
        [0.1563, 0.1463],
        [0.1566, 0.1463],
        [0.1623, 0.1530],
        [0.1608, 0.1532],
        [0.1624, 0.1523],
        [0.1566, 0.1467]